In [1]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate

from ipywidgets import AppLayout, FloatSlider,  IntText, VBox

In [2]:
class Parameters8:
    """
    A class to handle parameters for SIDARTHE calculations with 8 equations.
    """

    def __init__(self,
                 alpha: float = 0.15,
                 beta: float = 0.008443847910426,
                 gamma: float = 0.15,
                 epsilon: float = 0.0,
                 zeta: float = 0.079018779922852,
                 theta: float = 0.198057106656095,
                 kappa: float = 0.056288791238505,
                 my_lambda: float = 0.059610666036892,
                 mu: float = 0.007998690003693 + 0.005027246740356,
                 sigma: float = 0.007998690003693 /
                 (0.007998690003693 + 0.005027246740356) * 0.037040181047319 +
                 0.005027246740356 /
                 (0.007998690003693 + 0.005027246740356) * 0.055231044651498,
                 tau: float = 0.007998690003693 /
                 (0.007998690003693 + 0.005027246740356) * 0.015874367480393 +
                 0.005027246740356 /
                 (0.007998690003693 + 0.005027246740356) * 0.024198833665235,
                 n_total: int = 83000000,
                 y0 = None,
                 y0_abs = [
                     82636256, 20581, 0, 8041, 41931, 11469, 276911, 4810
                 ]):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.epsilon = epsilon
        self.zeta = zeta
        self.theta = theta
        self.kappa = kappa
        self.my_lambda = my_lambda
        self.mu = mu
        self.sigma = sigma
        self.tau = tau
        self.n_total = n_total
        self.y0_abs = y0_abs
        if y0 is None:
            self.y0 = [x / self.n_total for x in self.y0_abs]
        else:
            self.y0 = y0


def sidarthe_system8(parameters: Parameters8):
    """doc"""
    alpha = parameters.alpha
    beta = parameters.beta
    gamma = parameters.gamma
    epsilon = parameters.epsilon
    theta = parameters.theta
    zeta = parameters.zeta
    my_lambda = parameters.my_lambda
    kappa = parameters.kappa
    mu = parameters.mu
    sigma = parameters.sigma
    tau = parameters.tau

    def dF(t, y) -> np.array:
        """doc"""
        return np.array([
            -y[0] * (alpha * y[1] + beta * (y[2] + y[4]) + gamma * y[3]),
            y[0] * (alpha * y[1] + beta * (y[2] + y[4]) + gamma * y[3]) -
            (epsilon + zeta + my_lambda) * y[1],
            epsilon * y[1] - (zeta + my_lambda) * y[2],
            zeta * y[1] - (theta + mu + kappa) * y[3],
            zeta * y[2] + theta * y[3] - (mu + kappa) * y[4],
            mu * (y[3] + y[4]) - (sigma + tau) * y[5],
            my_lambda * (y[1] + y[2]) + kappa * (y[3] + y[4]) + sigma * y[5],
            tau * y[5]
        ])

    return dF

In [17]:
def integrate_system(alpha, beta, gamma, epsilon, t_end):
    parameters = Parameters8(alpha=alpha, beta=beta, gamma=gamma, epsilon=epsilon)
    dF = sidarthe_system8(parameters)
    y0 = parameters.y0
    t = np.linspace(0, t_end, t_end+1)
    y = np.zeros((len(t), len(y0)))  # array for solution
    y[0, :] = y0
    
    r = integrate.ode(dF).set_integrator("dopri5")
    r.set_initial_value(y0, 0)

    for i in range(1, t.size):
        y[i, :] = r.integrate(t[i])  # get one more value, add it to the array
        if not r.successful():
            raise RuntimeError("Could not integrate")
    return t, y



def update_lines(change):
    t, y = integrate_system(
        alpha=alpha_widget.value,
        beta=beta_widget.value,
        gamma=gamma_widget.value,
        epsilon=epsilon_widget.value,
        t_end=t_end
        )
    for index, line in enumerate(lines):
        line[0].set_data(t, y[:, index])
    fig.canvas.draw()
    fig.canvas.flush_events()
    return None

In [21]:
plt.ioff()

alpha_widget = FloatSlider(description=r"$\alpha$", value=0.15, min=0, max=1, step=.01)
beta_widget = FloatSlider(description=r"$\beta$", value=0.008443847910426, min=0, max=1, step=.01)
gamma_widget = FloatSlider(description=r"$\gamma$", value=0.15, min=0, max=1, step=.01)
epsilon_widget = FloatSlider(description=r"$\epsilon$", value=0., min=0, max=1, step=.01)

slider_box = VBox([alpha_widget, beta_widget, gamma_widget, epsilon_widget])

t_end = 365

fig, axes = plt.subplots(nrows=2, ncols=4)
plt.subplots_adjust(top=0.92,
                        bottom=0.08,
                        left=0.10,
                        right=0.95,
                        hspace=.5,
                        wspace=.5)
ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7 = axes.flatten()

fig.canvas.header_visible = False
fig.canvas.layout.min_height = '400px'

t, y = integrate_system(
    alpha=alpha_widget.value,
    beta=beta_widget.value,
    gamma=gamma_widget.value,
    epsilon=epsilon_widget.value,
    t_end=t_end
    )

lines = []
lines.append(ax0.plot(t, y[:, 0]))
ax0.set_title("S")
ax0.set_ylim([0, 1])

lines.append(ax1.plot(t, y[:, 1]))
ax1.set_title("I")
ax1.set_ylim([0, 1])

lines.append(ax2.plot(t, y[:, 2]))
ax2.set_title("D")
ax2.set_ylim([0, 1])

lines.append(ax3.plot(t, y[:, 3]))
ax3.set_title("A")
ax3.set_ylim([0, 0.1])

lines.append(ax4.plot(t, y[:, 4]))
ax4.set_title("R")
ax4.set_ylim([0, 1])

lines.append(ax5.plot(t, y[:, 5]))
ax5.set_title("T")
ax5.set_ylim([0, 0.1])

lines.append(ax6.plot(t, y[:, 6]))
ax6.set_title("H")
ax6.set_ylim([0, 1])

lines.append(ax7.plot(t, y[:, 7]))
ax7.set_title("E")
ax7.set_ylim([0, 0.1])

alpha_widget.observe(update_lines, names="value")

AppLayout(
    center=fig.canvas,
    footer=slider_box,
    pane_heights=[0, 6, 1]
)

AppLayout(children=(VBox(children=(FloatSlider(value=0.15, description='$\\alpha$', max=1.0, step=0.01), Float…

In [3]:
plt.ioff()

slider = FloatSlider(
    orientation='horizontal',
    description='Factor:',
    value=1.0,
    min=0.02,
    max=2.0
)

slider.layout.margin = '0px 30% 0px 30%'
slider.layout.width = '40%'

fig = plt.figure()
fig.canvas.header_visible = False
fig.canvas.layout.min_height = '400px'
plt.title('Plotting: y=sin({} * x)'.format(slider.value))

x = np.linspace(0, 20, 500)

lines = plt.plot(x, np.sin(slider.value * x))

def update_lines(change):
    plt.title('Plotting: y=sin({} * x)'.format(change.new))
    lines[0].set_data(x, np.sin(change.new * x))
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_lines, names='value')

AppLayout(
    center=fig.canvas,
    footer=slider,
    pane_heights=[0, 6, 1]
)

AppLayout(children=(FloatSlider(value=1.0, description='Factor:', layout=Layout(grid_area='footer', margin='0p…